In [2]:
from datetime import datetime, timedelta
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from score_drives import get_drive_stats

In [3]:
# df = get_drive_stats(2009, 2019, exclude_playoffs=False,
#                      exclude_blowouts=False, dpa_scores=False,
#                      opponent_adjustment=False,
#                      data_path='/home/jovyan/work/data')

df = get_drive_stats(2009, 2019, exclude_playoffs=False,
                     exclude_blowouts=False, dpa_scores=False,
                     opponent_adjustment=False,
                     data_path='C:/Users/309342/Documents/backup/nfl-master/data')

2010122604.json '2010122604'
min() arg is an empty sequence dict_keys([])


In [4]:
df.head()

,away_final_score,away_team,defensive_team,drive_time,end_quarter,end_time,first_play_desc,game_id,home_final_score,home_score_diff_last_quarter,...,offensive_win,defensive_win,tie,1_quarter_time,2_quarter_time,3_quarter_time,4_quarter_time,5_quarter_time,offense_home,defense_home
0,10,TEN,TEN,1.733333,1,13.266667,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,2009091000,13,0,...,1,0,0,1.733333,0.0,0.0,0.0,0.0,True,False
1,10,TEN,PIT,1.866667,1,11.400000,(13:16) C.Johnson up the middle to TEN 2 for n...,2009091000,13,0,...,0,1,0,1.866667,0.0,0.0,0.0,0.0,False,True
2,10,TEN,TEN,3.066667,1,8.333333,(11:24) B.Roethlisberger pass short right to M...,2009091000,13,0,...,1,0,0,3.066667,0.0,0.0,0.0,0.0,True,False
3,10,TEN,PIT,1.600000,1,6.733333,(8:20) C.Johnson right end to TEN 43 for 32 ya...,2009091000,13,0,...,0,1,0,1.600000,0.0,0.0,0.0,0.0,False,True
4,10,TEN,TEN,1.916667,1,4.816667,(6:44) R.Mendenhall up the middle to PIT 28 fo...,2009091000,13,0,...,1,0,0,1.916667,0.0,0.0,0.0,0.0,True,False


In [5]:
df.columns

Index(['away_final_score', 'away_team', 'defensive_team', 'drive_time',
       'end_quarter', 'end_time', 'first_play_desc', 'game_id',
       'home_final_score', 'home_score_diff_last_quarter', 'home_team',
       'last_play_desc', 'n_plays', 'offensive_team', 'penalty_yards',
       'result', 'start_quarter', 'start_time', 'start_yard_line',
       'yards_gained', 'drive_id', 'season', 'game_date', 'day_of_week',
       'unique_game_flag', 'game_in_season', 'is_playoffs', 'expected_points',
       'offensive_points', 'is_touchdown', 'is_field_goal', 'is_score',
       'is_interception', 'is_fumble', 'dst_points', 'home_points',
       'away_points', 'offensive_team_score_end', 'defensive_team_score_end',
       'offensive_team_score_start', 'defensive_team_score_start',
       'home_score_end', 'away_score_end', 'home_score_start',
       'away_score_start', 'offensive_final_score', 'defensive_final_score',
       'offensive_win', 'defensive_win', 'tie', '1_quarter_time',
       '2_q

In [6]:
print(df.loc[df['season'] == 2009, 'game_id'].values[0])
print(df.loc[df['season'] == 2010, 'game_id'].values[0])

2009091000
2010090900


In [7]:
week_start = datetime.strptime('20090910', '%Y%m%d')
print(week_start.weekday())
week_end = week_start + timedelta(days=4)
print(week_end.weekday())

3
0


In [9]:
def training_features_pipeline(df, games_back=256):
    # Initialize beginning values.
    week_start = datetime.strptime('20100909', '%Y%m%d') # Thursday of first week in train
    week_end = week_start + timedelta(days=4) # Monday of first week in train
    current_season = 2010 # Season of first week in train
    while week_end <= datetime.now():
        slate = get_train_slate(df, week_start, week_end)
        ddf = get_drive_data(df, current_season, week_start, games_back)
        print(slate)
        print(ddf.head())
        return ddf
#         week_start += timedelta(days=7)
#         week_end += timedelta(days=7)
        
        
def get_train_slate(df, week_start, week_end):
    slate_list = []
    sdf = df.loc[
        (df['game_date'] >= week_start) & 
        (df['game_date'] <= week_end)
    ].copy()
    columns = [
        'game_id', 'home_team', 'away_team', 'home_final_score',
        'away_final_score'
    ]
    for row in sdf[columns].values:
        slate_list.append(tuple(row.tolist()))
    return set(slate_list)
    
    
def get_drive_data(df, current_season, week_start, games_back):
    start_this_season, end_this_season = get_start_end_this_season(
        df, current_season, week_start, games_back
    )
    start_last_season = get_start_last_season(
        df, current_season, games_back, end_this_season
    )
    this_season_mask = (
        (df['season'] == current_season) & 
        (df['game_in_season'] >= end_this_season) &
        (df['game_in_season'] <= start_this_season)
    )
    last_season_mask = (
        (df['season'] == current_season - 1) &
        (df['game_in_season'] >= start_last_season)
    )
    ddf = df.loc[(this_season_mask) | (last_season_mask)].copy()
    return ddf
    
        
def get_start_end_this_season(df, current_season, week_start, games_back):
    season_mask = df['season'] == current_season
    date_mask = df['game_date'] < week_start
    season_drives = df.loc[(season_mask) & (date_mask), 'game_id'].copy()
    n_this_season = season_drives.nunique()
    print(n_this_season)
    if n_this_season >= games_back:
        return games_back - n_this_season, n_this_season
    else:
        return 0, n_this_season


def get_start_last_season(df, current_season, games_back, n_this_season):
    if n_this_season >= games_back:
        return 268
    n_last_season = games_back - n_this_season + 11
    game_in_season = 267 - n_last_season + 1
    season_mask = df['season'] == current_season - 1
    game_in_season_mask = df['game_in_season'] == game_in_season
    print(game_in_season)
    game_date = df.loc[(season_mask) & (game_in_season_mask), 'game_date'].min()
    print(game_date)
    week_start = get_week_start(game_date)
    date_mask = df['game_date'] >= week_start
    in_season_start = df.loc[(season_mask) & (date_mask), 'game_in_season'].min()
    return in_season_start
    
    
def get_week_start(game_date):
    weekday = game_date.weekday()
    print(weekday)
    if weekday >= 3:
        td = weekday - 3
    else:
        td = 7 + weekday - 3
    return game_date - timedelta(days=td)

In [11]:
ddf = training_features_pipeline(df, games_back=256)

0
1
2009-09-10 00:00:00
3
{('2010091207', 'CHI', 'DET', 19, 14), ('2010091204', 'JAX', 'DEN', 24, 17), ('2010091300', 'NYJ', 'BAL', 9, 10), ('2010090900', 'NO', 'MIN', 14, 9), ('2010091200', 'TB', 'CLE', 17, 14), ('2010091209', 'PIT', 'ATL', 15, 9), ('2010091201', 'BUF', 'MIA', 10, 15), ('2010091208', 'LA', 'ARI', 13, 17), ('2010091211', 'SEA', 'SF', 31, 6), ('2010091301', 'KC', 'LAC', 21, 14), ('2010091210', 'PHI', 'GB', 20, 27), ('2010091212', 'WAS', 'DAL', 13, 7), ('2010091206', 'NYG', 'CAR', 31, 18), ('2010091203', 'HOU', 'IND', 34, 24), ('2010091202', 'NE', 'CIN', 38, 24), ('2010091205', 'TEN', 'OAK', 38, 13)}
   away_final_score away_team defensive_team  drive_time  end_quarter  \
0                10       TEN            TEN    1.733333            1   
1                10       TEN            PIT    1.866667            1   
2                10       TEN            TEN    3.066667            1   
3                10       TEN            PIT    1.600000            1   
4           

In [12]:
ddf.head()

,away_final_score,away_team,defensive_team,drive_time,end_quarter,end_time,first_play_desc,game_id,home_final_score,home_score_diff_last_quarter,...,offensive_win,defensive_win,tie,1_quarter_time,2_quarter_time,3_quarter_time,4_quarter_time,5_quarter_time,offense_home,defense_home
0,10,TEN,TEN,1.733333,1,13.266667,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,2009091000,13,0,...,1,0,0,1.733333,0.0,0.0,0.0,0.0,True,False
1,10,TEN,PIT,1.866667,1,11.400000,(13:16) C.Johnson up the middle to TEN 2 for n...,2009091000,13,0,...,0,1,0,1.866667,0.0,0.0,0.0,0.0,False,True
2,10,TEN,TEN,3.066667,1,8.333333,(11:24) B.Roethlisberger pass short right to M...,2009091000,13,0,...,1,0,0,3.066667,0.0,0.0,0.0,0.0,True,False
3,10,TEN,PIT,1.600000,1,6.733333,(8:20) C.Johnson right end to TEN 43 for 32 ya...,2009091000,13,0,...,0,1,0,1.600000,0.0,0.0,0.0,0.0,False,True
4,10,TEN,TEN,1.916667,1,4.816667,(6:44) R.Mendenhall up the middle to PIT 28 fo...,2009091000,13,0,...,1,0,0,1.916667,0.0,0.0,0.0,0.0,True,False


In [16]:
ddf.columns

Index(['away_final_score', 'away_team', 'defensive_team', 'drive_time',
       'end_quarter', 'end_time', 'first_play_desc', 'game_id',
       'home_final_score', 'home_score_diff_last_quarter', 'home_team',
       'last_play_desc', 'n_plays', 'offensive_team', 'penalty_yards',
       'result', 'start_quarter', 'start_time', 'start_yard_line',
       'yards_gained', 'drive_id', 'season', 'game_date', 'day_of_week',
       'unique_game_flag', 'game_in_season', 'is_playoffs', 'expected_points',
       'offensive_points', 'is_touchdown', 'is_field_goal', 'is_score',
       'is_interception', 'is_fumble', 'dst_points', 'home_points',
       'away_points', 'offensive_team_score_end', 'defensive_team_score_end',
       'offensive_team_score_start', 'defensive_team_score_start',
       'home_score_end', 'away_score_end', 'home_score_start',
       'away_score_start', 'offensive_final_score', 'defensive_final_score',
       'offensive_win', 'defensive_win', 'tie', '1_quarter_time',
       '2_q

In [ ]:
ddf['off']

In [164]:
agg1_columns = ['game_id', 'offensive_team', 'defensive_team', 'offense_home', 'defense_home']

gdf = ddf.groupby(agg1_columns, as_index=False).agg(
    {'offensive_final_score': 'mean', 'defensive_final_score': 'mean',
     'offensive_win': 'mean', 'defensive_win': 'mean', 'yards_gained': 'sum',
     'penalty_yards': 'sum', 'offensive_points': 'mean', 'is_touchdown': 'mean', 
     'is_score': 'mean', 'is_interception': 'mean', 'is_fumble': 'mean', 
     'drive_time': 'sum', '4_quarter_time': 'sum',
     'drive_id': 'count', 'game_date': 'max', 'dst_points': 'mean'}
)

rename_dict = {
    'offensive_final_score': 'offense_score',
    'defensive_final_score': 'defense_score',
    'offensive_points': 'ppd', 
    'is_touchdown': 'tdr', 
    'is_score': 'scr',
    'is_interception': 'intr', 
    'is_fumble': 'fmbr', 
    'drive_id': 'possession_count',
    'dst_points': 'dppd',
    'game_date': 'last_game'
}

gdf['total_yards'] = gdf['yards_gained'] + gdf['penalty_yards']

gdf = gdf.rename(rename_dict, axis=1)

In [165]:
gdf.head()

,game_id,offensive_team,defensive_team,offense_home,defense_home,offense_score,defense_score,offensive_win,defensive_win,yards_gained,...,tdr,scr,intr,fmbr,drive_time,4_quarter_time,possession_count,last_game,dppd,total_yards
0,2009091000,PIT,TEN,True,False,13,10,1,0,357,...,0.076923,0.230769,0.153846,0.076923,36.183333,9.083333,13,2009-09-10,0.000000,332
1,2009091000,TEN,PIT,False,True,10,13,0,1,320,...,0.083333,0.166667,0.083333,0.083333,28.350000,5.766667,12,2009-09-10,0.000000,337
2,2009091300,ATL,MIA,True,False,19,7,1,0,281,...,0.181818,0.363636,0.000000,0.000000,30.883333,7.283333,11,2009-09-13,0.000000,263
3,2009091300,MIA,ATL,False,True,7,19,0,1,259,...,0.090909,0.090909,0.090909,0.272727,29.116667,5.050000,11,2009-09-13,0.000000,249
4,2009091301,BAL,KC,True,False,38,24,1,0,501,...,0.454545,0.545455,0.090909,0.000000,39.816667,8.083333,11,2009-09-13,0.636364,477


In [166]:
opponent_transformations = {
    'offensive_opp_ppd': ('defensive_team', 'ppd'),
    'offensive_opp_tdr': ('defensive_team', 'tdr'),
    'offensive_opp_scr': ('defensive_team', 'scr'),
    'defensive_opp_ppd': ('offensive_team', 'ppd'),
    'defensive_opp_tdr': ('offensive_team', 'tdr'),
    'defensive_opp_scr': ('offensive_team', 'scr'),
}

In [167]:
for key, (group, variable) in opponent_transformations.items():
    gdf[key] = gdf.groupby(group)[variable].transform('mean')
    
gdf.head()

,game_id,offensive_team,defensive_team,offense_home,defense_home,offense_score,defense_score,offensive_win,defensive_win,yards_gained,...,possession_count,last_game,dppd,total_yards,offensive_opp_ppd,offensive_opp_tdr,offensive_opp_scr,defensive_opp_ppd,defensive_opp_tdr,defensive_opp_scr
0,2009091000,PIT,TEN,True,False,13,10,1,0,357,...,13,2009-09-10,0.000000,332,2.129638,0.255399,0.369638,1.895979,0.207823,0.354895
1,2009091000,TEN,PIT,False,True,10,13,0,1,320,...,12,2009-09-10,0.000000,337,1.635352,0.181771,0.302756,1.770671,0.190062,0.336808
2,2009091300,ATL,MIA,True,False,19,7,1,0,281,...,11,2009-09-13,0.000000,263,2.022433,0.211756,0.394342,2.008444,0.239945,0.353532
3,2009091300,MIA,ATL,False,True,7,19,0,1,259,...,11,2009-09-13,0.000000,249,1.825571,0.208485,0.333148,1.879546,0.212307,0.348018
4,2009091301,BAL,KC,True,False,38,24,1,0,501,...,11,2009-09-13,0.636364,477,1.989272,0.223609,0.366681,1.929129,0.224991,0.344479


In [168]:
gdf.columns

Index(['game_id', 'offensive_team', 'defensive_team', 'offense_home',
       'defense_home', 'offense_score', 'defense_score', 'offensive_win',
       'defensive_win', 'yards_gained', 'penalty_yards', 'ppd', 'tdr', 'scr',
       'intr', 'fmbr', 'drive_time', '4_quarter_time', 'possession_count',
       'last_game', 'dppd', 'total_yards', 'offensive_opp_ppd',
       'offensive_opp_tdr', 'offensive_opp_scr', 'defensive_opp_ppd',
       'defensive_opp_tdr', 'defensive_opp_scr'],
      dtype='object')

In [169]:
from functools import partial

percentile_75 = partial(np.percentile, q=75)
percentile_25 = partial(np.percentile, q=25)

In [170]:
percentile_75([1,2,3,3,80])

3.0

In [171]:
offdf = gdf.copy()

offense_range_columns = [
    'offense_score', 
    'defense_score', 
    'total_yards',
    'ppd', 
#     'tdr', 
#     'scr', 
    'intr', 
    'fmbr', 
    'drive_time', 
    '4_quarter_time',
    'possession_count', 
    'dppd', 
    'offensive_opp_ppd', 
#     'offensive_opp_tdr',
#     'offensive_opp_scr'
]

for column in offense_range_columns:
    offdf['%s_min' % column] = offdf[column].copy()
    offdf['%s_25' % column] = offdf[column].copy()
    offdf['%s_median' % column] = offdf[column].copy()
    offdf['%s_75' % column] = offdf[column].copy()
    offdf['%s_max' % column] = offdf[column].copy()

In [172]:
defdf = gdf.copy()

defense_range_columns = [
    'offense_score', 
    'defense_score', 
    'total_yards',
    'ppd', 
#     'tdr', 
#     'scr', 
    'intr', 
    'fmbr', 
    'drive_time', 
    '4_quarter_time',
    'possession_count', 
    'dppd', 
    'defensive_opp_ppd', 
#     'defensive_opp_tdr',
#     'defensive_opp_scr'
]

for column in defense_range_columns:
    defdf['%s_min' % column] = defdf[column].copy()
    defdf['%s_25' % column] = defdf[column].copy()
    defdf['%s_median' % column] = defdf[column].copy()
    defdf['%s_75' % column] = defdf[column].copy()
    defdf['%s_max' % column] = defdf[column].copy()

In [173]:
basic_args = {
    'offensive_win': 'mean', 'last_game': 'max', 
    'offensive_opp_ppd': 'mean', 'offensive_opp_tdr': 'mean',
    'offensive_opp_scr': 'mean', 'defensive_opp_ppd': 'mean'
}

In [180]:
offense_args = {}

for col in offdf.columns:
    if '_min' in col:
        offense_args[col] = 'min'
    if '_25' in col:
        offense_args[col] = percentile_25
    if '_median' in col:
        offense_args[col] = 'median'
    if '_75' in col:
        offense_args[col] = percentile_75
    if '_max' in col:
        offense_args[col] = 'max'

In [181]:
defense_args = {}

for col in defdf.columns:
    if '_min' in col:
        defense_args[col] = 'min'
    if '_25' in col:
        defense_args[col] = percentile_25
    if '_median' in col:
        defense_args[col] = 'median'
    if '_75' in col:
        defense_args[col] = percentile_75
    if '_max' in col:
        defense_args[col] = 'max'

In [182]:
offense_args.update(basic_args)
defense_args.update(basic_args)

In [183]:
offense_group_cols = ['offensive_team', 'offense_home']

offdf = offdf.groupby(offense_group_cols, as_index=False).agg(offense_args)

offdf.head()

,offensive_team,offense_home,offense_score_min,offense_score_25,offense_score_median,offense_score_75,offense_score_max,defense_score_min,defense_score_25,defense_score_median,...,offensive_opp_ppd_25,offensive_opp_ppd_median,offensive_opp_ppd_75,offensive_opp_ppd_max,offensive_win,last_game,offensive_opp_ppd,offensive_opp_tdr,offensive_opp_scr,defensive_opp_ppd
0,ARI,False,9,17.00,24.0,31.00,41,3,17.00,20.0,...,1.915001,2.090861,2.152099,2.407789,0.666667,2010-01-16,1.993082,0.232040,0.357850,1.911353
1,ARI,True,7,16.00,28.0,31.00,51,10,20.00,21.0,...,1.482503,1.631125,1.770195,2.090861,0.555556,2010-01-10,1.654323,0.189295,0.300997,1.911353
2,ATL,False,10,16.75,20.5,28.00,45,7,10.00,27.0,...,1.337796,1.495249,1.753973,2.276705,0.375000,2010-01-03,1.595310,0.182384,0.292388,2.008444
3,ATL,True,7,19.75,22.0,28.75,31,3,12.25,17.0,...,1.647828,1.718804,1.984804,2.022433,0.750000,2009-12-27,1.773881,0.197280,0.330446,2.008444
4,BAL,False,3,14.50,20.5,28.50,33,0,14.75,21.5,...,1.511912,1.633239,1.791148,1.994103,0.400000,2010-01-16,1.657625,0.185175,0.307800,1.929129


In [184]:
defense_group_cols = ['defensive_team', 'defense_home']

defdf = defdf.groupby(defense_group_cols, as_index=False).agg(defense_args)

defdf.head()

,defensive_team,defense_home,offense_score_min,offense_score_25,offense_score_median,offense_score_75,offense_score_max,defense_score_min,defense_score_25,defense_score_median,...,defensive_opp_ppd_25,defensive_opp_ppd_median,defensive_opp_ppd_75,defensive_opp_ppd_max,offensive_win,last_game,offensive_opp_ppd,offensive_opp_tdr,offensive_opp_scr,defensive_opp_ppd
0,ARI,False,3,17.00,20.0,24.00,45,9,17.00,24.0,...,1.346919,1.555081,1.770671,2.503406,0.333333,2010-01-16,1.765321,0.210530,0.307733,1.614624
1,ARI,True,10,20.00,21.0,33.00,45,7,16.00,28.0,...,1.528313,1.888738,2.331212,2.440868,0.444444,2010-01-10,1.765321,0.210530,0.307733,1.862214
2,ATL,False,7,10.00,27.0,34.25,37,10,16.75,20.5,...,1.596070,1.753177,2.093246,2.503406,0.625000,2010-01-03,1.825571,0.208485,0.333148,1.817940
3,ATL,True,3,12.25,17.0,21.50,34,7,19.75,22.0,...,1.472532,1.586868,1.895360,2.503406,0.250000,2009-12-27,1.825571,0.208485,0.333148,1.678677
4,BAL,False,0,14.75,21.5,26.75,33,3,14.50,20.5,...,1.658470,2.235905,2.327301,2.440868,0.600000,2010-01-16,1.334428,0.137516,0.261454,1.971188


In [185]:
defdf.head().T

,0,1,2,3,4
defensive_team,ARI,ARI,ATL,ATL,BAL
defense_home,False,True,False,True,False
offense_score_min,3,10,7,3,0
offense_score_25,17,20,10,12.25,14.75
offense_score_median,20,21,27,17,21.5
offense_score_75,24,33,34.25,21.5,26.75
offense_score_max,45,45,37,34,33
defense_score_min,9,7,10,7,3
defense_score_25,17,16,16.75,19.75,14.5
defense_score_median,24,28,20.5,22,20.5
